# fetch_and_normalize.ipynb

Notebook này dùng để:
- Lấy dữ liệu OHLCV từ **Finnhub** và **Twelve Data**
- Chuẩn hoá về dạng:
  `datetime, Open, High, Low, Close, Volume, symbol, interval, source`
- Lưu vào thư mục: `input/<source>_<symbol>_<interval>/data.csv`

Yêu cầu:
- Có file `.env` ở root project, nội dung:
  ```env
  FINNHUB_API_KEY=your_finnhub_key_here
  TWELVEDATA_API_KEY=your_twelvedata_key_here


In [1]:
import os, requests
import pandas as pd
from datetime import datetime, timedelta, timezone
from dateutil import parser as date_parser
from dotenv import load_dotenv

load_dotenv()

FINNHUB_API_KEY = os.environ.get("FINNHUB_API_KEY")
TWELVEDATA_API_KEY = os.environ.get("TWELVEDATA_API_KEY")

print(FINNHUB_API_KEY != None, TWELVEDATA_API_KEY != None)

BASE_INPUT_DIR = "input"

True True


In [2]:
COLUMNS = ["Open", "High", "Low", "Close", "Volume"]

def save_df(df: pd.DataFrame, symbol, interval, source):
    path = os.path.join(BASE_INPUT_DIR, symbol + "_" + interval + "_" + source + ".csv")
    df = df.sort_values("datetime")
    df["datetime"] = pd.to_datetime(df["datetime"], utc=True)
    df.set_index("datetime", inplace=True)
    df = df[COLUMNS]
    df.to_csv(path, index_label="datetime")
    print(f"Saved: {path}")

# save_df(df, "IBM", "5m", "FINNHUB")

In [3]:
def fetch_finnhub_data(symbol: str, resolution: str, start_time: datetime, end_time: datetime) -> pd.DataFrame:
    url = "https://finnhub.io/api/v1/stock/candle"
    params = {
        "symbol": symbol,
        "resolution": resolution,
        "from": int(start_time.replace(tzinfo=timezone.utc).timestamp()),
        "to": int(end_time.replace(tzinfo=timezone.utc).timestamp()),
        "token": FINNHUB_API_KEY,
    }

    resp = requests.get(url, params=params)
    resp.raise_for_status()
    data = resp.json()

    if data.get("s") != "ok":
        raise ValueError(f"Finnhub error for {symbol} ({resolution}): {data}")

    df = pd.DataFrame({
        "datetime": [
            datetime.fromtimestamp(ts, tz=timezone.utc) for ts in data["t"]
        ],
        "Open": data["o"],
        "High": data["h"],
        "Low": data["l"],
        "Close": data["c"],
        "Volume": data["v"],
    })

    return df

'''
def fetch_twelvedata_data(symbol: str, interval: str, start_time: datetime, end_time: datetime) -> pd.DataFrame:
    url = "https://api.twelvedata.com/time_series"

    params = {
        "symbol": symbol,
        "interval": interval,
        "apikey": TWELVEDATA_API_KEY,
        "start_date": start_time.strftime("%Y-%m-%d %H:%M:%S"),
        "end_date": end_time.strftime("%Y-%m-%d %H:%M:%S"),
        "order": "asc",
        "timezone": "UTC",
    }

    resp = requests.get(url, params=params)
    resp.raise_for_status()
    data = resp.json()

    if "values" not in data:
        raise ValueError(f"Twelve Data error for {symbol} ({interval}): {data}")

    values = data["values"]
    df_raw = pd.DataFrame(values)

    df = pd.DataFrame()
    df["datetime"] = df_raw["datetime"].apply(
        lambda x: date_parser.parse(x).astimezone(timezone.utc)
    )

    df["Open"] = df_raw["open"].astype(float)
    df["High"] = df_raw["high"].astype(float)
    df["Low"] = df_raw["low"].astype(float)
    df["Close"] = df_raw["close"].astype(float)
    df["Volume"] = df_raw["volume"].astype(float)

    return df
'''
import numpy as np 
def fetch_twelvedata_data(symbol: str, interval: str,
                          start_time: datetime, end_time: datetime) -> pd.DataFrame:
    url = "https://api.twelvedata.com/time_series"

    params = {
        "symbol": symbol,
        "interval": interval,
        "apikey": TWELVEDATA_API_KEY,
        "start_date": start_time.strftime("%Y-%m-%d %H:%M:%S"),
        "end_date": end_time.strftime("%Y-%m-%d %H:%M:%S"),
        "order": "asc",
        "timezone": "UTC",
    }

    resp = requests.get(url, params=params)
    resp.raise_for_status()
    data = resp.json()

    if "values" not in data:
        raise ValueError(f"Twelve Data error for {symbol} ({interval}): {data}")

    df_raw = pd.DataFrame(data["values"])

    # debug thử xem API trả những cột gì
    print("TwelveData columns:", df_raw.columns.tolist())

    df = pd.DataFrame()
    df["datetime"] = df_raw["datetime"].apply(
        lambda x: date_parser.parse(x).astimezone(timezone.utc)
    )

    df["Open"]  = df_raw["open"].astype(float)
    df["High"]  = df_raw["high"].astype(float)
    df["Low"]   = df_raw["low"].astype(float)
    df["Close"] = df_raw["close"].astype(float)

    # Xử lý Volume mềm dẻo
    if "volume" in df_raw.columns:
        df["Volume"] = df_raw["volume"].astype(float)
    else:
        # Nếu không có volume, bạn chọn 1 trong 2:
        # 1) Gán 0.0
        df["Volume"] = 0.0

        # 2) hoặc gán NaN: df["Volume"] = np.nan
        # tuỳ cách bạn muốn validate / xử lý sau này

    return df

In [4]:
def map_finnhub_resolution_to_interval(resolution: str) -> str:
    """
    Map resolution Finnhub -> interval chuẩn.

    Finnhub: 1,5,15,30,60,D,W,M
    Chuẩn dùng: 1m,5m,15m,30m,1h,1d,1w,1mo
    """
    mapping = {
        "1": "1m",
        "5": "5m",
        "15": "15m",
        "30": "30m",
        "60": "1h",
        "D": "1d",
        "W": "1w",
        "M": "1mo",
    }
    return mapping.get(resolution, resolution)

def map_twelvedata_interval_to_standard(interval: str) -> str:
    """
    Map interval Twelve Data -> interval chuẩn.

    Twelve Data: 1min,5min,15min,30min,1h,4h,1day,1week,1month
    Chuẩn:       1m,5m,15m,30m,1h,4h,1d,1w,1mo
    """
    mapping = {
        "1min": "1m",
        "5min": "5m",
        "15min": "15m",
        "30min": "30m",
        "45min": "45m",
        "1h": "1h",
        "2h": "2h",
        "4h": "4h",
        "1day": "1d",
        "1week": "1w",
        "1month": "1mo",
    }
    return mapping.get(interval, interval)

def savedata_finnhub(symbol, resolution, days: int = 365):
    end_time = datetime.now(timezone.utc)
    start_time = end_time - timedelta(days=days)

    df = fetch_finnhub_data(symbol, resolution, start_time, end_time)
    interval = map_finnhub_resolution_to_interval(resolution)
    save_df(df, symbol, interval, "finnhub")

temp = 1
def savedata_twelvedata(symbol, interval, days: int = 365):
    end_time = datetime.now(timezone.utc)
    start_time = end_time - timedelta(days=days)

    df = fetch_twelvedata_data(symbol, interval, start_time, end_time)
    global temp
    temp = df.copy()
    print(df)
    std_interval = map_twelvedata_interval_to_standard(interval)
    save_df(df, symbol, std_interval, "twelvedata")

In [8]:
# savedata_finnhub(symbol="AAPL", resolution="D", days=365 * 2) # Cần premium?
savedata_twelvedata(symbol="AAPL", interval="1h", days=30)

TwelveData columns: ['datetime', 'open', 'high', 'low', 'close', 'volume']
                     datetime        Open        High         Low       Close  \
0   2025-10-28 06:30:00+00:00  269.135010  269.870000  268.250000  268.649990   
1   2025-10-28 07:30:00+00:00  268.642300  269.140010  268.149990  268.697200   
2   2025-10-28 08:30:00+00:00  268.695010  269.075806  268.350010  268.995000   
3   2025-10-28 09:30:00+00:00  268.980010  269.870000  268.820010  269.495000   
4   2025-10-28 10:30:00+00:00  269.495000  269.495000  268.799990  269.090210   
..                        ...         ...         ...         ...         ...   
149 2025-11-26 09:30:00+00:00  279.029999  279.530000  278.870000  279.079010   
150 2025-11-26 10:30:00+00:00  279.070007  279.290010  278.095001  278.549990   
151 2025-11-26 11:30:00+00:00  278.560000  278.581790  277.924990  278.260010   
152 2025-11-26 12:30:00+00:00  278.260100  278.940000  278.010010  278.079987   
153 2025-11-26 13:30:00+00:00  278